In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# Grabbing all csvs and storing them in dataframes

In [ ]:
consumption = pd.read_csv('../../Data/Fish/fish-and-seafood-consumption-per-capita.csv')
production = pd.read_csv('../../Data/Fish/fish-seafood-production.csv')
population = pd.read_csv('../../Data/Fish/population.csv')
life_expect = pd.read_csv('../../Data/to_merge/life_expectancy.csv')
c_n_c = pd.read_csv('../../Data/to_merge/country_and_continents.csv')

# Doing some renaming for the merges
Production and consumption had overly-long names, cleaning those up. Also adjust the population df to contain only relevent years.

In [ ]:
production = production.rename(columns={'Fish and seafood | 00002960 || Production | 005511 || tonnes': 'Production_in_tons'})

In [ ]:
consumption = consumption.rename(columns={'Fish and seafood | 00002960 || Food available for consumption | 0645pc || kilograms per year per capita': 'kg_per_yr_per_capita'})

In [ ]:
population = population[population['Year'] > 1960].reset_index(drop=True)

# Beginning to merge
Adding one df at a time and ensuring that the integrity of the data remains consistant

in the case of life_expect, i had to pivot the table and give it a variable column name 'Year', then i had to convert the numerical values into workable datatye I.E int, float.
then I merged it with merged_df and dropped the redundant names.

In [ ]:
columns_to_merge = ['Year', 'Code', 'Production_in_tons']
merged_df = consumption.merge(production[columns_to_merge], on=['Year', 'Code'])

In [ ]:
columns_to_merge = ['Code', 'Year','Population (historical estimates)', 'Entity']
merged_df = merged_df.merge(population[columns_to_merge], on=['Code', 'Year', 'Entity'])

In [ ]:
to_drop = ['Country Name', 'Country Code'] # redundant columns

melted_life_expect = life_expect.melt(id_vars=['Country Name', 'Country Code'], var_name='Year', value_name='Value')
melted_life_expect['Year'] = melted_life_expect['Year'].astype(int)
melted_life_expect['Value'] = melted_life_expect['Value'].astype(float)
merged_df = merged_df.merge(melted_life_expect, left_on=['Entity', 'Code', 'Year'], right_on=['Country Name', 'Country Code', 'Year'], how='left')
merged_df.drop(to_drop, inplace=True, axis=1)
merged_df = merged_df.rename(columns={'Value': 'Life_Expectancy_years'})

In [ ]:
c_n_c = c_n_c[['country _or_area', 'continent']]
c_n_c.rename(columns={'country _or_area': 'Entity'}, inplace=True)
merged_df = merged_df.merge(c_n_c, on=['Entity'])

# In depth analysis and cleaning

In [231]:
merged_df

,Entity,Code,Year,kg_per_yr_per_capita,Production_in_tons,Population (historical estimates),Life_Expectancy_years,continent
0,Afghanistan,AFG,1961,0.030559,300.0,8790140,32.96,Asia
1,Afghanistan,AFG,1962,0.030578,300.0,8969055,33.47,Asia
2,Afghanistan,AFG,1963,0.030598,300.0,9157463,33.97,Asia
3,Afghanistan,AFG,1964,0.030614,300.0,9355510,34.46,Asia
4,Afghanistan,AFG,1965,0.030627,300.0,9565154,34.95,Asia
...,...,...,...,...,...,...,...,...
9305,Zimbabwe,ZWE,2016,3.679239,25800.0,14452705,60.29,Africa
9306,Zimbabwe,ZWE,2017,3.397224,26800.0,14751101,60.81,Africa
9307,Zimbabwe,ZWE,2018,3.117559,32300.0,15052191,61.20,Africa
9308,Zimbabwe,ZWE,2019,2.870986,29230.0,15354606,61.49,Africa


In [ ]:
us_df = merged_df[merged_df['Code'] == 'USA'].reset_index(drop=True)

In [ ]:
us_df

# Writing new df to final CSV

In [233]:
merged_df.to_csv('../../Data/final_csv_all_data.csv', index=False)